In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from tabulate import tabulate
from diniz_fts import *

Using TensorFlow backend.


Load dataset:

In [2]:
df = pd.read_csv('./dataset_.csv')
X = df.values
X = X.astype('float32')

Visualization of variables:

In [3]:
# X is composed of the 4 input variables shifted at time t=120, as described in the article,
# therefore, each line of X is composed of 480 samples (120x4) + 2 columns representing the output.
# The output (clogging or non-clogging) is represented by [1 0] and [0 1], respectively.
# now, we will present the 4 input variables and the output, as well as general parameters.
Xp = pd.DataFrame(X)
inputs = pd.DataFrame({'Casting Spd':Xp[0],'Mold Lvl':Xp[1],'Gate Opn.':Xp[2],'Tundish':Xp[3]})
outputs = pd.DataFrame({'Clogging':Xp[Xp.shape[1]-2], 'Normal':Xp[Xp.shape[1]-1]})

inputs.describe()


,Casting Spd,Mold Lvl,Gate Opn.,Tundish
count,3.054426e+06,3.054426e+06,3.054426e+06,3.054426e+06
mean,1.188411e+00,1.007837e+02,7.063924e+01,5.513210e+00
std,2.208997e-01,8.715239e+00,4.749716e+00,4.998255e-01
min,0.000000e+00,3.416350e+01,-2.888620e-02,5.000000e+00
25%,1.008350e+00,9.938640e+01,6.753560e+01,5.000000e+00
50%,1.180000e+00,1.000180e+02,7.087970e+01,6.000000e+00
75%,1.350000e+00,1.006580e+02,7.377410e+01,6.000000e+00
max,1.710000e+00,2.000000e+02,1.023190e+02,6.000000e+00


In [4]:
inputs.head()

,Casting Spd,Mold Lvl,Gate Opn.,Tundish
0,1.20000,100.542000,70.076103,5.0
1,1.19254,100.589996,68.787804,5.0
2,1.19725,100.617996,68.397697,5.0
3,1.19314,100.646004,69.981903,5.0
4,1.19000,101.015999,69.698196,5.0


In [5]:
# Presentation of the % of samples classified as Clogging and Normal:
n_clog = outputs.Clogging[outputs.Clogging == 1].sum()  
n_normal = outputs.Normal[outputs.Normal == 1].sum()  

table = [['Output', 'Count', '%'], 
         ['Clogging', int(n_clog),  round(100*n_clog/(n_clog+n_normal))], 
         ['Non-Clogging', int(n_normal), round(100*n_normal/(n_clog+n_normal))]]
print(tabulate(table))

------------  -------  --
Output        Count    %
Clogging      203186   7
Non-Clogging  2851240  93
------------  -------  --


In [6]:
outputs.head()

,Clogging,Normal
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0


In [7]:
del df, Xp, table, inputs, n_clog, n_normal, outputs

Determine parameters for model training (as reported in the article - ConvLSTM #7):

In [8]:
delay = 120 #determines the temporal sample size of each input matrix (as explained in the manuscript)

n_features = 4 #number of input variables (as explained in the manuscript)

# defines the parameters used in the network
ftr_ly_01 = 256 #first layer filters
ftr_ly_02 = 16 #second layer filters
krn_ly_01 = 5 #first layer kernels
krn_ly_02 = 3 #second layer kernels
dropout_01 = 0.5 #first dropout layer
dense_ly = 200 #dense layer neurons
dropout_02 = 0.3 #second dropout layer

# defines the lists that will receive the results of the performance criteria of each repetition of the code
accuracy_ = list() #to save the accuracy values for each round
precision_ = list() #to save the precision values for each round
Recall_ = list() #to save the recall values for each round
f1_ = list() #to save the F1 values for each round
MCC_ = list() #to save the MCC values for each round


Neural network training:

In [9]:
# defines the number of times the network will be retrained/tested (a kind of k-fold, with k=5)    
rounds = 5

for r in range(rounds):
    
    # Perform neural network training and store network test performance
    print('ConvLstm network training - round = %d' % (r+1))
    accuracy_test, precision_test, recall_test, f1_test, MCC_test = traintest_convlstm(X, r, delay, n_features, ftr_ly_01, ftr_ly_02, krn_ly_01, krn_ly_02, dropout_01, dense_ly, dropout_02)
    
    print('Performance parameters of the ConvLstm network (test) - round = %d' % (r+1))
    # presents the results of the test number "r" of the ConvLSTM(256,16) network:
    summarize_results_point(accuracy_test, precision_test, recall_test, f1_test, MCC_test)
        
    # groups the results of each network test
    accuracy_.append(accuracy_test)
    precision_.append(precision_test)
    Recall_.append(recall_test)
    f1_.append(f1_test)
    MCC_.append(MCC_test)  
    

ConvLstm network training - round = 1
Model: "ConvLSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_1 (ConvLSTM2D)  (None, 4, 1, 26, 256)     1332224   
_________________________________________________________________
conv_lst_m2d_2 (ConvLSTM2D)  (None, 1, 24, 16)         52288     
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 24, 16)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 384)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               77000     
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)     

Presents the final results after testing the ConvLSTM(256,16) network:

In [11]:
summarize_results(accuracy_, precision_, Recall_, f1_, MCC_)

Accuracy 	  	 	 	 	 95.098% (+/-0.223)
Precision 	 	 	 	 	 95.677% (+/-0.335)
Recall 	 	 	 	 	 	 97.244% (+/-0.364)
F1 	 	 	 	 	 	 96.410% (+/-0.202)
MCC 	 	 	 	 	 	 0.903 (+/-0.005)


Post-processing heuristics:

In [12]:
#1 - separates train, valid, test_complete dataset

# X_train - dataset of inputs used for training the network
# y_train - dataset of outputs used in training the network
# X_valid - dataset of inputs used for validation during network training
# y_valid - dataset of outputs used for validation in network training
# X_test - dataset of inputs used for network tests
# y_test - dataset of outputs used for network testing

X_train, y_train, X_valid, y_valid, X_test, y_test = split_dataset(X, delay, n_features)

In [13]:
#2 - train a convlstm network with X_train, y_train, X_valid, y_valid and return the model
model = convlstm(X_train, y_train, X_valid, y_valid, delay, n_features, ftr_ly_01, ftr_ly_02, krn_ly_01, krn_ly_02, dropout_01, dense_ly, dropout_02)   


Model: "ConvLSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_11 (ConvLSTM2D) (None, 4, 1, 26, 256)     1332224   
_________________________________________________________________
conv_lst_m2d_12 (ConvLSTM2D) (None, 1, 24, 16)         52288     
_________________________________________________________________
dropout_11 (Dropout)         (None, 1, 24, 16)         0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 384)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 200)               77000     
_________________________________________________________________
dropout_12 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 2)                 402

In [17]:
# 2.1 - save the model (optional)
# model.save('convlstm_save.h5')

# 2.2 - load the model (optional)
# model = tf.keras.models.load_model('convlstm_save.h5')

In [18]:
# 3 - apply test set to the network with heuristics
accuracy_t, precision_t, recall_t, f1_t, MCC_t, accuracy_h, precision_h, recall_h, f1_h, MCC_h = convlstm_heuristic(X_test, y_test, model)

40578/40578 [==============================] - 11s 273us/sample


Overall ConvLSTM 7 Result before post-processing heuristics:

In [21]:
print('Before Heuristic:')
summarize_results_point(accuracy_t, precision_t, recall_t, f1_t, MCC_t)

Before Heuristic:
Accuracy 	  	 	 	 	 96.165%
Precision 	 	 	 	 	 95.767%
Recall 	 	 	 	 	 	 97.906%
F1 	 	 	 	 	 	 96.268%
MCC 	 	 	 	 	 	 0.925


Overall ConvLSTM 7 Result after post-processing heuristics:

In [22]:
print('After Heuristic:')
summarize_results_point(accuracy_h, precision_h, recall_h, f1_h, MCC_h)

After Heuristic:
Accuracy 	  	 	 	 	 99.869%
Precision 	 	 	 	 	 97.938%
Recall 	 	 	 	 	 	 98.616%
F1 	 	 	 	 	 	 98.276%
MCC 	 	 	 	 	 	 0.982
